#Note
This is a clear sky model for zenith opacity; it does not include
* Clouds and fog
* Rain (BAD)

In [47]:
# %load ../Atmosphere/opacity.py
def calc_atmospheric_opacity(T, RH, P, h, f):
    """ 
        Calculates zenith opacity according to ITU-R P.676-9. For elevations > 10 deg.
        Use as "Tsky*(1-exp(-opacity/sin(el)))" for elevation dependence.
        T: temperature in deg C
        RH: relative humidity, 0 < RH < 1
        P: dry air pressure in hPa (equiv. mbar)  # WE MEASURE TOTAL: DRY=TOTAL-WET
        h: height above sea level in km
        f: frequency in GHz (must be < 55 GHz)
        This function returns the return: approximate atmospheric opacity at zenith [Nepers]
    """
    es = 6.1121*np.exp((18.678-T/234.5)*T/(257.14+T)) # [hPa] from A. L. Buck research manual 1996
    rho = RH*es*216.7/(T+273.15) # [g/m^3] from A. L. Buck research manual 1996 (ITU-R ommited the factor "RH" - a mistake)
    #print es, rho
    # The following is taken directly from ITU-R P.676-9
    """
    p_tot = P + es # from eq 3
    """
    Pdry = P-es
    
    rho = rho*np.exp(h/2) # Adjust to sea level as per eq 32
    print  "water vapour pressure",es,"mbar ; density" ,rho ,"g/m**3"
    # eq 22
    r_t = 288./(273.+T)
    r_p = P/1013.
    phi = lambda a, b, c, d: r_p**a*r_t**b*np.exp(c*(1-r_p)+d*(1-r_t))
    E_1 = phi(0.0717,-1.8132,0.0156,-1.6515)
    E_2 = phi(0.5146,-4.6368,-0.1921,-5.7416)
    E_3 = phi(0.3414,-6.5851,0.2130,-8.5854)
    #print E_1,E_2,E_3
    #print r_t,r_p,f
    # Following is valid only for f <= 54 GHz
    yo = ( 7.2*r_t**2.8 / (f**2+0.34*r_p**2*r_t**1.6) + 0.62*E_3 / ((54-f)**(1.16*E_1)+0.83*E_2) )\
        * f**2 * r_p**2 *1e-3
    #print "YO", yo
    # eq 23
    """
    End of dry part
    """
    n_1 = 0.955*r_p*r_t**0.68 + 0.006*rho
    n_2 = 0.735*r_p*r_t**0.5 + 0.0353*r_t**4*rho
    g = lambda f, f_i: 1+(f-f_i)**2/(f+f_i)**2
    yw = (  3.98*n_1*np.exp(2.23*(1-r_t))/((f-22.235)**2+9.42*n_1**2)*g(f,22)\
          + 11.96*n_1*np.exp(0.7*(1-r_t))/((f-183.31)**2+11.14*n_1**2)\
          + 0.081*n_1*np.exp(6.44*(1-r_t))/((f-321.226)**2+6.29*n_1**2)\
          + 3.66*n_1*np.exp(1.6*(1-r_t))/((f-325.153)**2+9.22*n_1**2)\
          + 25.37*n_1*np.exp(1.09*(1-r_t))/(f-380)**2\
          + 17.4*n_1*np.exp(1.46*(1-r_t))/(f-448)**2\
          + (844.6*n_1*np.exp(0.17*(1-r_t))/(f-557)**2)*g(f,557)
          + (290*n_1*np.exp(0.41*(1-r_t))/(f-752)**2)*g(f,752)
          + (8.3328e4*n_2*np.exp(0.99*(1-r_t))/(f-1780.)**2)*g(f,1780)
          ) * f**2*r_t**2.5*rho*1e-4
    
    # eq 25
    t_1 = 4.64/(1+0.066*r_p**-2.3) * np.exp(-((f-59.7)/(2.87+12.4*np.exp(-7.9*r_p)))**2)
    t_2 = 0.14*np.exp(2.12*r_p) / ((f-118.75)**2+0.031*np.exp(2.2*r_p))
    t_3 = 0.0114/(1+0.14*r_p**-2.6) * f * (-0.0247+0.0001*f+1.61e-6*f**2) / (1-0.0169*f+4.1e-5*f**2+3.2e-7*f**3)
    ho = 6.1/(1+0.17*r_p**-1.1)*(1+t_1+t_2+t_3)
    #print yo,yw, "dB/km dry and wet"
    #print ho ,"Dry scale height"
    # eq 26
    sigma_w = 1.013/(1+np.exp(-8.6*(r_p-0.57)))
    hw = 1.66*( 1 + 1.39*sigma_w/((f-22.235)**2+2.56*sigma_w)\
               + 3.37*sigma_w/((f-183.31)**2+4.69*sigma_w) + 1.58*sigma_w/((f-325.1)**2+2.89*sigma_w) )
    #print hw, "Wet scale height"
    # Attenuation from dry & wet atmosphere relative to a point outside of the atmosphere
    #A = yo*ho*np.exp(-h/ho) + yw*hw*np.exp(-h/hw) # [dB] from equations 27, 30 & 31
    A= yo*ho +yw*hw # Equation 27
    # See also 
    #print A, "dB\n", 10**(-0.1*A), "Attenuation factor"
    return 0.1*A/np.log10(np.e)  # Convert dB to Nepers



In [48]:
import katdal
import numpy as np
filename = '/var/kat/archive/data/RTS/telescope_products/2015/07/18/1437231715.h5'
h5 = katdal.open(filename,centre_freq=12500.0e6)
temp=h5.sensor['Enviro/air_temperature'].mean()  #Degrees C
press=h5.sensor['Enviro/air_pressure'].mean()    # millibar
humid=h5.sensor['Enviro/air_relative_humidity'].mean()/100.  #relative humidity in range 0-1

print "air_temperature", temp
print "air_pressure",press
print "air_relative_humidity",humid
height=h5.ants[0].position_wgs84[2]/1000. # height in km
freq=h5.channel_freqs[1:].mean()/1e9      # freq in GHz
print "Height", height
print "Freq", freq, "GHz"
tau = calc_atmospheric_opacity(temp,humid,press,height,freq)
print tau
print "Attenuation ", np.exp(-tau), "or ", (1- np.exp(-tau))*100,"percent"
print "Noise from sky about", 275*(1-np.exp(-tau)),"K"  # not ground temperature, we need about 3km up

air_temperature 10.8464427191
air_pressure 899.257278231
air_relative_humidity 0.641648885643
Height 1.03811873316
Freq 12.5 GHz
water vapour pressure 12.9924562579 mbar ; density 10.6895478954 g/m**3
0.0147684182741
Attenuation  0.985340099944 or  1.46599000559 percent
Noise from sky about 4.03147251537 K


In [49]:
print tau

0.0147684182741


#Compare with GBT 
note on dynamic scheduling DS Project Note 5.2
J. J. Condon & Dana S. Balser
GBT Archive: DS005.2

* Fig 1 Zeith opacity below 50GHz. 
$\tau$ values at 12GHz about 0.02neper for T=288K (15C) 55% cloud cover 
* see also http://www.gb.nrao.edu/~rmaddale/Weather/opacity.html#OpacityTime12

In [50]:
x=calc_atmospheric_opacity(10.,0.63,899.,1.038,12.5)
print x
print np.exp(-x)

water vapour pressure 12.2786016958 mbar ; density 9.94788239801 g/m**3
0.0143114479005
0.985790474074


In [51]:
x=calc_atmospheric_opacity(33.,0.25,890.,1.038,1.5)
print x
print np.exp(-x)

water vapour pressure 50.3332868346 mbar ; density 14.9664494682 g/m**3
0.00507980195529
0.99493307842


In [52]:
def watervap(T,humid):
    """
    calculates water vapour pressure in hectopascal  given a temperature and humidity
    T in Celsius
    humid is fractional humidity 
    
    check at 100C and 1.0 expect 1013mbar
    """
    w=6.1121*np.exp((18.678-T/234.5)*T/(257.14+T))
    return w*humid

In [53]:
def vaisala(T,humid):
    """
    calculates water vapour pressure in hectopascal a temperature and humidity
    T in Celsius
    humid is fractional humidity 
    valid from -20 to +50C
    check at 100C and 1.0 expect 1013mbar
    """
    A=6.116441 
    m=7.591386 
    Tn=240.7263 
    p=A*10**((m*T)/(T+Tn))
    return p*humid
    
    

In [54]:
print watervap(100,1.0)
print watervap(30,0.33), vaisala(30,0.33)

1013.07780897
14.0089148636 14.0035185265


In [55]:
def h0(r_p,f):
    """
    equivalent height for dry air for earth-space at zenith
    r_p  is fractional pressure (P/1013)
    f is in GHz
    Equation 25a-e
    valid if f<70
    and h0 <0.7*r_p**0.3  - which is true up tp high altitude
    """
    t_1= 4.64*np.exp( -((f-59.7)/(2.87+12.4*np.exp(-7.9*r_p)))**2)
    t_2= 0.14*np.exp(2.12*r_p)/((f-118.75)**2+0.031*np.exp(2.2*r_p))
    t_3=(-0.0247 + 0.0001*f + 1.61e-6*f**2)/(1-0.0169*f + 4.1e-5*f**2 + 3.2e-7*f**3)
    t_3=0.0114*f*t_3/(1+0.14*r_p)
    h=6.1*(1+t_1+ t_2+ t_3)/(1+0.17*r_p**(-1.1)) # usually about 5-6
    return h


In [56]:
def hw(r_p,f):
    """
    equivalent height for wet air for earth-space at zenith
    r_p  is fractional pressure (P/1013)
    f is in GHz
    Equation 26a-b
    valid if f<350GHz
    """
    s=1.013/(1+np.exp(-8.6*(r_p- 0.57)))
    t1=1.39*s/((f-22.235)**2 +2.56*s)
    t2=3.37*s/((f-183.31)**2 +4.69*s)
    t3=1.58*s/((f-325.1)**2  +2.89*s)
    h=1.66*(1+t1+t2+t3)  # first term dominates -answer always near 1.6
    return h


In [57]:
print hw(.8,14.) , h0(1.,1.1)

1.68949193935 5.2126777634


In [58]:
def hs(f):
    """
    Simpler version from www2.widener.edu/~rpj0001/courses/ENGR647/ClassNotes/LECT11.pdf
    OK for 1<f<56.7
    """
    h=5.386-3.332734e-2*f + 1.87185e-3*f**2 - 3.52087e-5*f**3 + 83.26/((f-60)**2+1.2)
    return h

In [59]:
print hs(1.4), hs(14)

5.36715152742 5.22901269148


In [60]:
def hw2(f):
    """
    Simpler version from www2.widener.edu/~rpj0001/courses/ENGR647/ClassNotes/LECT11.pdf
    OK for 1< 350
    """
    h=1.65*(1 + 1.61/((f-22.3)**2+2.91) + 3.33/((f-183.3)**2 +4.58) + 1.90/((f-325.1)**2 +3.24) )
    return h


In [61]:
print hw2(12.5), hw(1.0,12.5)
print hs(12.5), h0(1.0,12.5)

1.67706728789 1.68365768746
5.23000014329 5.19521537781


#Cloud model
http://www2.widener.edu/~rpj0001/courses/ENGR647/ClassNotes/LECT11.pdf 

## Cloud densities from wikipedia
in g/m**3
* cirrus 0.03
* fog 0.05
* stratus or cumulus 0.25-0.30
* stratocumulus 0.45
* cumulonimbus 1-3
## thickness
 stratocumulus water path 20-80 g/m**2 giving thickness about 0.1 -0.4 km


In [69]:
# need to know cloud thickness 

def cloudab(f,T,l,m):
    """
    f in GHz
    T in celsius
    l in km
    m is water density in the cloud g/m**3
    returns attenuation in dB
    """
    theta =300/(273+T)
    # dielctric constants
    e0=77.6 + 103.3*(theta-1)
    e1=5.48
    e2=3.51
    #
    #Frequencies
    fp= 20.09 - 142*(theta-1) + 294*(theta-1)**2
    fs= 590- 1500*(theta-1)
    # Complex permittivity
    e_2= f*(e0-e1)/(fp*(1+(f/fp)**2)) + f*(e1-e2)/(fs*(1+(f/fs)**2))
    e_1 = (e0-e1)/(fp*(1+(f/fp)**2)) + (e1-e2)/(fs*(1+(f/fs)**2)) +e2  
    #
    eta=(2+e_1)/e_2
    k=0.819*f/(e_2*(1+eta**2))
    attenuation=k*m*l #dB
    return attenuation 

In [72]:
x= cloudab(12.5, 20, 0.4, 0.25)
factor= 10**(-0.1*x)
print x, factor

0.0297451065 0.9931743375


#Rain
see 
https://www.itu.int/dms_pubrec/itu-r/rec/p/R-REC-P.838-3-200503-I!!PDF-E.pdf

80.0